# markdown.obisidian.personal.index_notes
> Functions for managing index notes one's Obsidian.md math vault.

In a Obsidian math vault, it is convenient to keep index notes, which list links to other index notes or standard information notes. 

The methods in this module (semi)automatically 
- create (standard information) notes in appropriate folders,
- set up the notes,
- add links of the notes to appropriate index notes
- add to both the index note and the standard information note a footnote indicating where the content of the information note originates from in the original text, e.g.  
`[[foag_transition_function_for_a_vector_bundle_over_a_manifold]], 13.1.1`, `[[bredon_tensor_product_of_graded_groups]], Page 315`, 
`[[eilenberg_zilber_theorem|Eilenberg Zilber theorem]], Theorem 1.3, Corollary 1.4`

In [ ]:
#| default_exp markdown.obsidian.personal.index_notes

In [ ]:
#| export
import glob
import os
from os import PathLike
from pathlib import Path
import re
from typing import Union

from natsort import natsorted

from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import heading_title
from trouver.markdown.obsidian.links import (
    find_links_in_markdown_text, ObsidianLink, links_from_text
)
from trouver.markdown.obsidian.vault import (
    VaultNote, note_name_unique, note_path_by_name
)

In [ ]:
from unittest import mock
from fastcore.test import *

## Automatically filling in some notes

- I need to identify the names of subsections/subchapters inside a chapter, and identify the correspondence of subdirectories in the directory with headings in the index note.
- I need to use regex to find Theorems/Corollaries/Propositions/Lemmas/Definitions/Remarks/Examples (I'll refere to these as Numberings) in LaTeX code that has been OCR'd
- For each subsection/subchapter, I need to collect these Numberings, create an information note for each one, and add the number/page number.

In [ ]:
#| export
def subsections_listed_in_index_note(
        index_note: Union[VaultNote, str], # The index note
        vault: PathLike
        ) -> dict[Union[int, str], [dict, str]]: # The keys are 1. line numbers and 2. `'title'`. The values are dict and str (the blank str if root node), respectively.
    """
    Return subsections/subchapters as listed in the index note

    **See Also**
    
    - The `get_headings_tree` function of the `MarkdownFile` class.
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    mf_file = MarkdownFile.from_vault_note(index_note)
    return mf_file.get_headings_tree()

In [ ]:
text = r"""# 1. Some section title
- [[some_note]], Page 1
- [[some_note_2]], Page 2

# 2. Some other section title
- [[some_note_3]], Page 2
- [[some_note_4]], Page 3

# 3. Section 3
- [[some_note_5]], Page 3

# 4. Section 4
# 5. Section 5
"""

with mock.patch("trouver.markdown.markdown.file.open", mock.mock_open(read_data=text)):
    fake_vn = VaultNote(rel_path='fake_note.md', vault='')  # Think of this as a VaultNote object whose underlying file has `text` as its content.
    subsections_in_text = subsections_listed_in_index_note(fake_vn, vault='')
    expected_output = {
        'title': '',
        0: {'title': '# 1. Some section title'},
        4: {'title': '# 2. Some other section title'},
        8: {'title': '# 3. Section 3'},
        11: {'title': '# 4. Section 4'},
        12: {'title': '# 5. Section 5'}}
    test_eq(subsections_in_text, expected_output)

The following is closer to an example in practice:

In [ ]:
text = r"""# 18.1. (Desired) properties of cohomology
- [ ] [[foag_ 18.1|foag_Desired_properties_of_cohomology]],  18.1, Page 465
# 18.2. Definitions and proofs of key properties
- [ ] [[foag_Definitions 18.2]], Definitions 18.2, Page 470
# 18.3. Cohomology of line bundles on projective space
- [ ] [[foag_Cohomology 18.3|]], Cohomology 18.3, Page 475
# 18.4. Riemann-Roch, degrees of coherent sheaves, and arithmetic genus
- [ ] [[foag_Riemann 18.4]], Riemann 18.4, Page 477
# 18.5. A first glimpse of Serre duality
- [ ] [[foag_A 18.5]], A 18.5, Page 485
# 18.6. Hilbert functions, Hilbert polynomials, and genus
- [ ] [[foag_Hilbert 18.6|foag_Hilbert_function_of_a_coherent_sheaf_on_a_projective_k_scheme]], Hilbert 18.6, Page 488
# 18.7. ⋆ Serre's cohomological characterization of ampleness
- [ ] [[foag_ 18.7|]],  18.7, Page 494
# 18.8. Higher pushforward (or direct image) sheaves
- [ ] [[foag_Higher 18.8]], Higher 18.8, Page 497
# 18.9. ⋆ From projective to proper hypotheses: Chow's Lemma and Grothendieck’s Coherence Theorem
- [ ] [[foag_ 18.9]],  18.9, Page 501
"""

with mock.patch("trouver.markdown.markdown.file.open", mock.mock_open(read_data=text)):
    mock_vault = Path('')
    mock_vn = VaultNote(rel_path='mock_note.md', vault=mock_vault)
    subsections_in_text = subsections_listed_in_index_note(mock_vn, vault=mock_vault)
    expected_output = {
        'title': '',
        0: {'title': '# 18.1. (Desired) properties of cohomology'},
        2: {'title': '# 18.2. Definitions and proofs of key properties'},
        4: {'title': '# 18.3. Cohomology of line bundles on projective space'},
        6: {'title': '# 18.4. Riemann-Roch, degrees of coherent sheaves, and arithmetic genus'},
        8: {'title': '# 18.5. A first glimpse of Serre duality'},
        10: {'title': '# 18.6. Hilbert functions, Hilbert polynomials, and genus'},
        12: {'title': "# 18.7. ⋆ Serre's cohomological characterization of ampleness"},
        14: {'title': '# 18.8. Higher pushforward (or direct image) sheaves'},
        16: {'title': "# 18.9. ⋆ From projective to proper hypotheses: Chow's Lemma and Grothendieck’s Coherence Theorem"}}
    test_eq(subsections_in_text, expected_output)


In [ ]:
#| export
def subsection_folders(
        index_note: Union[VaultNote, str], # The index note
        vault: PathLike,
        output_type: str, # `'absolute_path'`, `'relative_path'`, or `'name'`
        ) -> list[str]: # List of immediate subdirectories in the directory containing the index note.
    """
    Return subdirectories corresponding to subsections/subchapters, i.e.
    the folders in the same directory as the index note.

    The folders are arranged in the order specified by `natsorted`.
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    parent_directory = (vault / index_note.rel_path).parent
    # print(str(parent_directory))
    glob_result = natsorted(glob.glob(str(parent_directory) + '/**/'))
    if output_type == 'absolute_path':
        return glob_result
    elif output_type == 'relative_path':
        return [str(Path(dir).relative_to(vault)) for dir in glob_result]
    elif output_type == 'name':
        return [Path(dir).name for dir in glob_result]

In [ ]:
mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('mock_reference_folder') / Path('mock_chapter')
folders = [  #glob.glob would return the folders in this order, at least on Windows:
    '1 section',
    '10 section',
    '11 section',
    '2 section',
    '3 section',
    '4 section',
    '5 section',
    '6 section',
    '7 section',
    '8 section',
    '9 section']

mock_glob_return_value = [str(mock_path / folder) for folder in folders]

with mock.patch("__main__.glob.glob", return_value=mock_glob_return_value):
    mock_index_note = VaultNote(rel_path='_index_mock_chapter.md', vault= mock_vault)
    
    sample_output_absolute_path = subsection_folders(mock_index_note, mock_vault, output_type='absolute_path')
    test_shuffled(sample_output_absolute_path, mock_glob_return_value)
    test_eq(sample_output_absolute_path, natsorted(mock_glob_return_value))

    sample_output_relative_path = subsection_folders(mock_index_note, mock_vault, output_type='relative_path')
    expected_output_for_relative_paths = [os.path.relpath(folder, mock_vault) for folder in mock_glob_return_value]
    test_shuffled(sample_output_relative_path, expected_output_for_relative_paths)
    test_eq(sample_output_relative_path, natsorted(expected_output_for_relative_paths))

    # test_eq(sample_output_absolute_path, )
    sample_output_name = subsection_folders(mock_index_note, mock_vault, output_type='name')
    test_shuffled(sample_output_name, folders)
    test_eq(sample_output_name, natsorted(folders))

The following example is closer to an example in practice:

In [ ]:
# Test an example that looks closer to what would happen in practice.
mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('algebraic_geometry') / Path('foag') / Path('chapter_18_cech_cohomology_of_quasicoherent_sheaves')
folders = ['181_desired_properties_of_cohomology',
    '182_definitions_and_proofs_of_key_properties',
    '183_cohomology_of_line_bundles_on_projective_space',
    '184_riemann_roch_degrees_of_coherent_sheaves_and_arithmetic_genus',
    '185_a_first_glimpse_of_serre_duality',
    '186_hilbert_functions_hilbert_polynomials_and_genus',
    '187_â‹†_serres_cohomological_characterization_of_ampleness',
    '188_higher_pushforward_or_direct_image_sheaves',
    '189_â‹†_from_projective_to_proper_hypotheses_chows_lemma_and_grothendieckâ€™s_coherence_theorem']
mock_glob_return_value = [str(mock_path / folder) for folder in folders]

with mock.patch("__main__.glob.glob", return_value=mock_glob_return_value):
    mock_index_note = VaultNote(rel_path='_index_chapter_18_cech_cohomology_of_quasicoherent_sheaves.md', vault= mock_vault)
    
    sample_output_absolute_path = subsection_folders(mock_index_note, mock_vault, output_type='absolute_path')
    test_eq(sample_output_absolute_path, mock_glob_return_value)
    sample_output_relative_path = subsection_folders(mock_index_note, mock_vault, output_type='relative_path')
    expected_output_for_relative_paths = [os.path.relpath(folder, mock_vault) for folder in mock_glob_return_value]
    test_eq(sample_output_relative_path, expected_output_for_relative_paths)
    sample_output_name = subsection_folders(mock_index_note, mock_vault, output_type='name')
    test_eq(sample_output_name, folders)

## Corresponding headings in index notes and subfolders

In [ ]:
#| export 
def get_alphanumeric(
        title: str, # The title of either a folder or a heading. Must start with an alphanumeric.
        title_type: str # Either `folder` or `heading`.
        ) -> str: # An alphabet or a numeric (arabic or roman)
    """
    Get the alphanumeric of a title of either a folder or a heading
    in an index noteh.

    Assumes that each folder is titled
    `'{alphanumeric}_{folder_title}'` and each heading is titled
    `'{alphanumeric}. {heading_title}'`
    """
    assert title_type in ['folder', 'heading']
    if title_type == 'folder':
        return re.sub(r'(.*?)_.*' , r'\1', title)
    else:
        return re.sub(r'(.*?)\. .*', r'\1', title)
    


In [ ]:
test_eq(get_alphanumeric('1. Higher direct images', 'heading'), '1')
test_eq(get_alphanumeric('1_higher_direct_images', 'folder'), '1')
test_eq(get_alphanumeric('12_higher_direct_images_the_leray_spectral_sequence', 'folder'), '12')
test_eq(get_alphanumeric('VII_elliptic_curves_over_local_fields', 'folder'), 'VII')
test_eq(get_alphanumeric('A_properties_of_morphisms', 'folder'), 'A')

In [ ]:
#| export 
def correspond_headings_with_folder(
        index_note: VaultNote, vault: PathLike,
        include_non_heading: bool = True) -> dict[str, str]:
    """
    Return tuples of corresponding headings in an index note
    with folder names.
    
    Assumes that each folder is titled
    `'{alphanumeric}_{folder_title}'` and each heading is titled
    `'{alphanumeric}. {heading_title}'`
    
    **Parameters**
    - index_note - VaultNote
    - vault - PathLike
    - include_non_heading - bool
        - If `True`, and if there is text before any heading, then treat
        such text as being under a "blank" heading. Defaults to `True`.
    
    **Returns**
    - dict[str, str]
        - Each key is a str indexing the headings and folders. The keys
        are usually alphanumerics (arabic or roman), depending on the
        numbering system of chapters/sections of the reference/text.
        The values are tuples `(folder_title, heading_title)` without 
        the alphanumeric. For the blank heading, the key/index, the folder title,
        and the heading title are all the empty str.
    """
    index = MarkdownFile.from_vault_note(index_note)
    headings = index.get_headings(levels=1)
    headings = [heading_title(heading) for heading in headings]
    folders = subsection_folders(index_note, vault, output_type='name')
    correspond_dict = {get_alphanumeric(heading, 'heading'): (heading, folder)
                       for heading, folder in zip(headings, folders)}
    # TODO do a better job at the conditional below; 
    # for example, consider the start of the text blank if it's just empty lines with spaces.
    if (include_non_heading and index.parts
            and index.parts[0]['type'] != MarkdownLineEnum.HEADING):
        correspond_dict[''] = ('', '')
    return correspond_dict
    

In [ ]:
mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('algebraic_geometry') / Path('foag') / Path('chapter_18_cech_cohomology_of_quasicoherent_sheaves')
folders = ['181_desired_properties_of_cohomology',
    '182_definitions_and_proofs_of_key_properties',
    '183_cohomology_of_line_bundles_on_projective_space',
    '184_riemann_roch_degrees_of_coherent_sheaves_and_arithmetic_genus',
    '185_a_first_glimpse_of_serre_duality',
    '186_hilbert_functions_hilbert_polynomials_and_genus',
    '187_â‹†_serres_cohomological_characterization_of_ampleness',
    '188_higher_pushforward_or_direct_image_sheaves',
    '189_â‹†_from_projective_to_proper_hypotheses_chows_lemma_and_grothendieckâ€™s_coherence_theorem']
mock_glob_return_value = [str(mock_path / folder) for folder in folders]

text = r"""# 18.1. (Desired) properties of cohomology
- [ ] [[foag_ 18.1|foag_Desired_properties_of_cohomology]],  18.1, Page 465
# 18.2. Definitions and proofs of key properties
- [ ] [[foag_Definitions 18.2]], Definitions 18.2, Page 470
# 18.3. Cohomology of line bundles on projective space
- [ ] [[foag_Cohomology 18.3|]], Cohomology 18.3, Page 475
# 18.4. Riemann-Roch, degrees of coherent sheaves, and arithmetic genus
- [ ] [[foag_Riemann 18.4]], Riemann 18.4, Page 477
# 18.5. A first glimpse of Serre duality
- [ ] [[foag_A 18.5]], A 18.5, Page 485
# 18.6. Hilbert functions, Hilbert polynomials, and genus
- [ ] [[foag_Hilbert 18.6|foag_Hilbert_function_of_a_coherent_sheaf_on_a_projective_k_scheme]], Hilbert 18.6, Page 488
# 18.7. ⋆ Serre's cohomological characterization of ampleness
- [ ] [[foag_ 18.7|]],  18.7, Page 494
# 18.8. Higher pushforward (or direct image) sheaves
- [ ] [[foag_Higher 18.8]], Higher 18.8, Page 497
# 18.9. ⋆ From projective to proper hypotheses: Chow's Lemma and Grothendieck’s Coherence Theorem
- [ ] [[foag_ 18.9]],  18.9, Page 501
"""
mock_index_file = MarkdownFile.from_string(text)

with (mock.patch("__main__.glob.glob", return_value=mock_glob_return_value),
      mock.patch("trouver.markdown.markdown.file.MarkdownFile.from_vault_note", return_value=mock_index_file)):

    mock_index_note = VaultNote(rel_path = '_index_mock.md', vault=mock_vault)
    # subsections_listed_in_index_note(mock_index_note, vault=mock_vault)
    sample_output = correspond_headings_with_folder(mock_index_note, mock_vault)
    print(sample_output)
    test_eq(len(sample_output), 9)
    for key, value in sample_output.items():
        assert value[0].startswith(key)
        assert value[1].startswith(key.replace('.', ''))

{'18.1': ('18.1. (Desired) properties of cohomology', '181_desired_properties_of_cohomology'), '18.2': ('18.2. Definitions and proofs of key properties', '182_definitions_and_proofs_of_key_properties'), '18.3': ('18.3. Cohomology of line bundles on projective space', '183_cohomology_of_line_bundles_on_projective_space'), '18.4': ('18.4. Riemann-Roch, degrees of coherent sheaves, and arithmetic genus', '184_riemann_roch_degrees_of_coherent_sheaves_and_arithmetic_genus'), '18.5': ('18.5. A first glimpse of Serre duality', '185_a_first_glimpse_of_serre_duality'), '18.6': ('18.6. Hilbert functions, Hilbert polynomials, and genus', '186_hilbert_functions_hilbert_polynomials_and_genus'), '18.7': ("18.7. ⋆ Serre's cohomological characterization of ampleness", '187_â‹†_serres_cohomological_characterization_of_ampleness'), '18.8': ('18.8. Higher pushforward (or direct image) sheaves', '188_higher_pushforward_or_direct_image_sheaves'), '18.9': ("18.9. ⋆ From projective to proper hypotheses: Chow

## Move information notes to their appropriate folders.
Sometimes, I end up creating information notes in the wrong folders. It would be nice to detect which ones are in the wrong folders and to move them appropriately.

In [ ]:
#| export
def information_notes_linked_in_index_note(
        index_note: VaultNote, # The note indexing the information notes.
        vault: PathLike,
        hints: list[PathLike] = None # Hints on where the information notes are likely to be found at.  Each path is relative to `vault` and points to a directory. Defaults to `None`.
        ) -> dict[str, list[VaultNote]]: # Each key is the index for the heading (usually either an alphanumerical or a roman numerical). Each value is a list of the information notes linked in the index note.
    """Find information notes to be moved to the correct folder.
    
    Current implementation just looks at level 1 headings.
    This function is used in `move_information_notes_to_correct_folder`.
    Assumes that all notes in the vault have unique names.
    """
    parent_folder = os.path.dirname(index_note.rel_path)
    headings_folders = correspond_headings_with_folder(index_note, vault)
    mf = MarkdownFile.from_vault_note(index_note)
    headings_text = mf.get_headings_and_text(levels=1, include_start=True)
    headings_text = {heading_title(heading): text for heading, text
                     in headings_text.items()}
    text_under_headings = {heading_index: headings_text[heading] 
                            for heading_index, (heading, _) in headings_folders.items()}
    links_by_headings = {heading_index:links_from_text(text) for
                         heading_index, text in text_under_headings.items()}
    note_names_by_headings = {heading_index:[il.file_name for il in links] 
                              for heading_index, links in links_by_headings.items()}
    # Find notes by headings, but also pass the folder corresponding to the heading
    # as a hint of where to find the note for speedup, in case the note is 
    # already at the right place.
    folders_by_index = {heading_index: Path(vault) / parent_folder / heading_folder 
                        for heading_index, (_, heading_folder) in headings_folders.items()}
    if not hints:
        hints = []
    notes_by_headings = {heading_index: [VaultNote(vault, name=nn, hints=hints+[folders_by_index[heading_index]]) 
                                         for nn in note_names]
                         for heading_index, note_names in note_names_by_headings.items()}
    return notes_by_headings
    

In [ ]:
# TODO: finish example
VaultNote.clear_cache()

mock_vault = Path('mock_absolute_path')
# mock_path = 

text = r"""
- [[hi]]
- [[bye]]
- [[another note]]
- [[yet another note#anchor]]
"""


mock_index_file = MarkdownFile.from_string(text)

# with (mock.patch("trouver.markdown.markdown.file.MarkdownFile.from_vault_note", return_value=mock_index_file)):
#     mock_index_note = VaultNote(rel_path = '_index_note.md', vault=mock_vault)
#     sample_output = information_notes_linked_in_index_note(mock_index_note, mock_vault)
#     print(sample_output)

In [ ]:
# index_note = VaultNote(MATH_VAULT_LOCATION, 
#                        name='_index_29_the_lefschetz_fixed_point_formula_for_nonconstant_sheaves')
# information_notes_linked_in_index_note(index_note, MATH_VAULT_LOCATION, hints=['algebraic_geometry/foag/_temp_foag'])